Random Forest

Imports

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd


Import Dataset

In [1]:
DATASET_PATH = "../data/output/tfidf_dataset.csv"
RNG = 42  # Random seed

df = pd.read_csv(DATASET_PATH)

print(df['cyberbullying'].value_counts())
df.head(10)

NameError: name 'pd' is not defined

Fitting

In [ ]:

# Features and labels
X = df.drop(columns=["cyberbullying"])
y = df["cyberbullying"]

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Truncated SVD to reduce dimensionality
svd = TruncatedSVD(n_components=300, random_state=42)

# Random Forest
rf = RandomForestClassifier(random_state=42)

# Create pipeline
pipeline = Pipeline([
    ("svd", svd),
    ("rf", rf)
])

# Hyperparameter grid
param_grid = {
    "rf__n_estimators": [100, 200],
    "rf__max_depth": [None, 10, 20],
    "rf__min_samples_split": [2, 5],
    "rf__min_samples_leaf": [1, 3],
    "rf__class_weight": ["balanced"]
}

# Cross-validation setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Grid search
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    n_jobs=1  # safer for memory
)

# Fit model
print("Fitting model (this may take a while)...")
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

# Predictions
y_pred = best_model.predict(X_test)

MemoryError: Unable to allocate 1.55 GiB for an array with shape (10000, 20768) and data type float64

Evaluation Metrics

In [ ]:
# Evaluation
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Not Cyberbullying", "Cyberbullying"]))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
labels = ["Not Cyberbullying", "Cyberbullying"]
annot_labels = [
    [f"TN = {cm[0,0]}", f"FP = {cm[0,1]}"],
    [f"FN = {cm[1,0]}", f"TP = {cm[1,1]}"]
]

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=annot_labels, fmt='', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Improved Random Forest Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()